# Homework 02 - Data from the web

The first thing we have done was to analyze the requests on IS-Acadamia using Postman in order to understand how to create the requests needed.
It was found that by modifying the GPS field it was easily possible to perform the extraction.

The work has been separated as follow:
- Creation of a GPS list with section, year and seasons
- Extraction of the infos using those GPS
- Data clearing
- Filtering of data and visualization of results

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

## Creation of a list sorted by GPS

In [5]:
gps_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=&ww_x_PERIODE_ACAD=null&zz_x_PERIODE_PEDAGO=&ww_x_PERIODE_PEDAGO=null&zz_x_HIVERETE=&ww_x_HIVERETE=null&dummy=ok'

gps_r = requests.get(gps_url)

In [6]:
gps_html = bs(gps_r.content, "html.parser")

gps_html_ = gps_html.find_all(class_='ww_x_GPS')
gps_list = []
gps_list2 = []
for elem in gps_html_:
    gps_list.append(elem.string.split(','))
    #string s_ of format "loadReport('ww_x_GPS=24065660');return false;" shorten to "24065660" only
    s_ = elem['onclick'].split('=')[1].split("')")[0]
    gps_list2.append(s_)

In [7]:
gps_df = pd.concat((pd.DataFrame(gps_list), pd.DataFrame(gps_list2)), axis=1)
gps_df.columns = ["Section", "Year", "AcPeriod", "GPS"]
gps_df.set_index("GPS", inplace=True)
gps_df = gps_df.ix[1:]

In [8]:
gps_df.head(10) #only head shown for convenience

,Section,Year,AcPeriod
GPS,,,
24065660,Echange IN,2005-2006,Semestre automne
24065672,Echange IN,2005-2006,Semestre printemps
39467937,Echange IN,2006-2007,Semestre automne
39467957,Echange IN,2006-2007,Semestre printemps
71454914,Echange IN,2007-2008,Semestre automne
71454935,Echange IN,2007-2008,Semestre printemps
117663512,Echange IN,2008-2009,Semestre automne
117663545,Echange IN,2008-2009,Semestre printemps
230302571,Echange IN,2009-2010,Semestre automne


## Extracting the infos using GPS

Reminder: We will focus exclusively on the academic unit Informatique.

Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6. Compute how many months it took each student to go from the first to the sixth semester. Partition the data between male and female students, and compute the average -- is the difference in average statistically significant?

Perform a similar operation to what described above, this time for Master students. Notice that this data is more tricky, as there are many missing records in the IS-Academia database. Therefore, try to guess how much time a master student spent at EPFL by at least checking the distance in months between Master semestre 1 and Master semestre 2. If the Mineur field is not empty, the student should also appear registered in Master semestre 3. Last but not the least, don't forget to check if the student has an entry also in the Projet Master tables. Once you can handle well this data, compute the "average stay at EPFL" for master students. Now extract all the students with a Spécialisation and compute the "average stay" per each category of that attribute -- compared to the general average, can you find any specialization for which the difference in average is statistically significant?

BONUS: perform the gender-based study also on the Master students, as explained in 1. Use scatterplots to visually identify changes over time. Plot males and females with different colors -- can you spot different trends that match the results of your statistical tests?


In [9]:
years = range(2007,2016)
years_ = []
for e in years:
    years_.append(" {0}-{1}".format(str(e),str(e+1)))

In [10]:
bachelors = []
for i in range(1,7):
    bachelors.append(" Bachelor semestre {}".format(i))

In [13]:
gps_final = gps_df[gps_df['Section'] == "Informatique"][gps_df['Year'].isin(years_)][gps_df['AcPeriod'].isin(bachelors)]
gps_final.head()

C:\Users\Sylvain\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,Section,Year,AcPeriod
GPS,,,
71297531,Informatique,2007-2008,Bachelor semestre 1
71297550,Informatique,2007-2008,Bachelor semestre 2
39494788,Informatique,2007-2008,Bachelor semestre 3
39494823,Informatique,2007-2008,Bachelor semestre 4
19561980,Informatique,2007-2008,Bachelor semestre 5


In [47]:
url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
dfs_complete = []
dfs_simple = []
for gps_value in gps_final.index:
    payload = {'ww_x_GPS': gps_value, 'ww_i_reportModel': 133685247, 'ww_i_reportModelXsl' : 133685270}
    req = requests.get(url, params=payload)
    df = pd.read_html(req.content, header=1)[0]
    df.set_index("No Sciper", inplace=True)
    df_complete = df.rename(columns = {'Statut':'Status' + str(gps_final.loc[gps_value].AcPeriod) + ' ' + str(gps_final.loc[gps_value].Year)})
    df_simple = df.rename(columns = {'Statut':'Status' + str(gps_final.loc[gps_value].AcPeriod)})
    dfs_complete.append(df_complete)
    dfs_simple.append(df_simple)

In [48]:
dfBA_simple = dfs_simple[0]
for df in dfs_simple:
    dfBA_simple = dfBA_simple.combine_first(df)

In [49]:
dfBA_complete = dfs_complete[0]
for df in dfs_complete:
    dfBA_complete = dfBA_complete.combine_first(df)

In [50]:
dfBA_simple = dfBA_simple[dfBA_simple['Status Bachelor semestre 1'].notnull()]
dfBA_simple = dfBA_simple[dfBA_simple['Status Bachelor semestre 6'].notnull()]

In [62]:
dfBA_completetemp = dfBA_complete
dfBA_completetemp = dfBA_completetemp.loc[dfBA_simple.index]
dfBA_completetemp["Nb_Semesters"] = dfBA_complete.ix[:,8:62].notnull().sum(axis=1)

In [65]:
dfBA_completetemp[["Nom Prénom", "Nb_Semesters"]]

,Nom Prénom,Nb_Semesters
No Sciper,,
147008,Good Xavier,6
169569,Arévalo Christian,6
169731,Knecht Mathieu,8
169795,Scheiben Pascal,9
171195,Richter Arnaud,6
171619,Buchschacher Nicolas,6
174905,Aubelle Flavien,10
175190,Hanser Valérian,7
175280,Uberti Quentin,8
